In [99]:
from pyspark.sql import functions as F
from pyspark.sql import SparkSession

In [100]:
# 1. Iniciar sesión de Sparkw
spark = SparkSession.builder.appName("ClasificacionUrbanoRural").getOrCreate()

In [102]:
# 2. Cargar el dataset de población y acceso a internet
df_poblacion = spark.read.csv('data/Internet_Fijo_Penetraci_n_Municipio.csv', header=True, inferSchema=True, encoding='UTF-8')

In [103]:
# 3. Cargar los datasets de áreas de terreno (por departamentos)
df_areas_municipio = spark.read.csv('data/deptos/*.csv', header=True, inferSchema=True, encoding='UTF-8')

In [104]:
# 4. Convertir los códigos de municipio y departamento a string para hacer coincidir los formatos
df_poblacion = df_poblacion.withColumn('COD_MUNICIPIO', F.col('COD_MUNICIPIO').cast('string')) \
    .withColumn('COD_DEPARTAMENTO', F.col('COD_DEPARTAMENTO').cast('string'))

df_areas_municipio = df_areas_municipio.withColumn('MUNICIPIO', F.col('MUNICIPIO').cast('string')) \
    .withColumn('DEPARTAMENTO', F.col('DEPARTAMENTO').cast('string'))

In [105]:
# 5. Sumar las áreas por municipio (usando MUNICIPIO)
df_areas_municipio = df_areas_municipio.groupBy('DEPARTAMENTO', 'MUNICIPIO').agg(F.sum('AREA_TERRENO').alias('AREA_TOTAL_KM2'))


In [106]:
# 6. Realizar el LEFT JOIN utilizando COD_MUNICIPIO de df_poblacion y MUNICIPIO de df_areas_municipio
df_completo = df_poblacion.join(df_areas_municipio, df_poblacion['COD_MUNICIPIO'] == df_areas_municipio['MUNICIPIO'], how='left')


In [107]:
# 7. Eliminar las columnas `DEPARTAMENTO` y `MUNICIPIO` duplicadas de df_areas_municipio
df_completo = df_completo.drop(df_areas_municipio['DEPARTAMENTO']).drop(df_areas_municipio['MUNICIPIO'])


In [108]:
# 8. Verificar si el área total (AREA_TOTAL_KM2) sigue teniendo valores nulos
df_completo.filter(F.col('AREA_TOTAL_KM2').isNull()).show()


+----+---------+----------------+------------+-------------+-------------+----------------------------+--------------+------+--------------+
| AÑO|TRIMESTRE|COD_DEPARTAMENTO|DEPARTAMENTO|COD_MUNICIPIO|    MUNICIPIO|No. ACCESOS FIJOS A INTERNET|POBLACIÓN DANE|INDICE|AREA_TOTAL_KM2|
+----+---------+----------------+------------+-------------+-------------+----------------------------+--------------+------+--------------+
|2021|        4|               8|   ATLANTICO|         8433|      MALAMBO|                       13293|        142095|0,0936|          NULL|
|2022|        1|               8|   ATLANTICO|         8433|      MALAMBO|                       13299|        143781|0,0925|          NULL|
|2021|        1|               8|   ATLANTICO|         8433|      MALAMBO|                       12614|        142095|0,0888|          NULL|
|2022|        2|               8|   ATLANTICO|         8433|      MALAMBO|                       13036|        143781|0,0907|          NULL|
|2021|       

In [109]:
# 9. Calcular el promedio del área por departamento para rellenar los valores nulos
df_departamentos_mean = df_areas_municipio.groupBy('DEPARTAMENTO').agg(F.mean('AREA_TOTAL_KM2').alias('mean_area'))


In [110]:
# 10. Unir el promedio de áreas al dataset principal
df_completo = df_completo.join(df_departamentos_mean.withColumnRenamed("DEPARTAMENTO", "COD_DEPARTAMENTO"),
                               on="COD_DEPARTAMENTO", how="left")

In [111]:
# 11. Rellenar los valores nulos en AREA_TOTAL_KM2 con el promedio del departamento (mean_area)
df_completo = df_completo.withColumn(
    'AREA_TOTAL_KM2',
    F.when(F.col('AREA_TOTAL_KM2').isNull(), F.col('mean_area')).otherwise(F.col('AREA_TOTAL_KM2'))
)

In [112]:
# 12. Asegurarse de que no queden nulos en mean_area o AREA_TOTAL_KM2
df_completo = df_completo.fillna({'AREA_TOTAL_KM2': 0, 'mean_area': 0})

In [113]:
# 13. Calcular la densidad poblacional
df_completo = df_completo.withColumn('densidad_poblacional', F.col('POBLACIÓN DANE') / F.col('AREA_TOTAL_KM2'))

In [114]:
# 14. Asegurarse de que la densidad poblacional tenga valores
df_completo = df_completo.fillna({'densidad_poblacional': 0})

In [115]:
# 15. Clasificar como Urbano o Rural según la densidad poblacional (ajustado para rangos más precisos)
df_completo = df_completo.withColumn(
    'Clasificacion',
    F.when(F.col('densidad_poblacional') >= 100, 'Urbano')
    .when((F.col('densidad_poblacional') > 0) & (F.col('densidad_poblacional') < 100), 'Rural')
    .otherwise('Sin clasificación')
)

In [117]:
# 16. Verificar si quedan valores nulos en todo el dataset
df_completo.select([F.count(F.when(F.col(f"`{c}`").isNull(), c)).alias(c) for c in df_completo.columns]).show()

+----------------+---+---------+------------+-------------+---------+----------------------------+--------------+------+--------------+---------+--------------------+-------------+
|COD_DEPARTAMENTO|AÑO|TRIMESTRE|DEPARTAMENTO|COD_MUNICIPIO|MUNICIPIO|No. ACCESOS FIJOS A INTERNET|POBLACIÓN DANE|INDICE|AREA_TOTAL_KM2|mean_area|densidad_poblacional|Clasificacion|
+----------------+---+---------+------------+-------------+---------+----------------------------+--------------+------+--------------+---------+--------------------+-------------+
|               0|  0|        0|           0|            0|        0|                           0|             0|     0|             0|        0|                   0|            0|
+----------------+---+---------+------------+-------------+---------+----------------------------+--------------+------+--------------+---------+--------------------+-------------+



In [118]:
# 17. Guardar el dataset transformado como un archivo CSV
output_path = "data/deptos/transformaciones/df_completo_exportado.csv"

In [119]:
# Combinar todas las particiones en un solo archivo CSV
df_completo.coalesce(1).write.csv(output_path, header=True, mode='overwrite')

In [121]:
# Contar cuántos municipios han sido clasificados como "Urbano"
num_urbanos = df_completo.filter(F.col('Clasificacion') == 'Urbano').count()

In [122]:
# Mostrar el resultado
print(f"El número total de municipios clasificados como 'Urbano' es: {num_urbanos}")


El número total de municipios clasificados como 'Urbano' es: 0


In [123]:
# Verificar si aún hay valores nulos o 0 en AREA_TOTAL_KM2
df_completo.filter((F.col('AREA_TOTAL_KM2').isNull()) | (F.col('AREA_TOTAL_KM2') == 0)).show()


+----------------+----+---------+------------+-------------+------------+----------------------------+--------------+------+--------------+---------+--------------------+-----------------+
|COD_DEPARTAMENTO| AÑO|TRIMESTRE|DEPARTAMENTO|COD_MUNICIPIO|   MUNICIPIO|No. ACCESOS FIJOS A INTERNET|POBLACIÓN DANE|INDICE|AREA_TOTAL_KM2|mean_area|densidad_poblacional|    Clasificacion|
+----------------+----+---------+------------+-------------+------------+----------------------------+--------------+------+--------------+---------+--------------------+-----------------+
|              11|2020|        3|      BOGOTA|        11001|BOGOTA, D.C.|                     2069115|       7743955|0,2672|           0.0|      0.0|                 0.0|Sin clasificación|
|              11|2020|        1|      BOGOTA|        11001|BOGOTA, D.C.|                     1957333|       7743955|0,2528|           0.0|      0.0|                 0.0|Sin clasificación|
|              11|2019|        4|      BOGOTA|        1

In [124]:
# Mostrar algunos valores de densidad poblacional y los municipios correspondientes
df_completo.select('COD_MUNICIPIO', 'MUNICIPIO', 'POBLACIÓN DANE', 'AREA_TOTAL_KM2', 'densidad_poblacional').show(20)


+-------------+------------+--------------+--------------------+--------------------+
|COD_MUNICIPIO|   MUNICIPIO|POBLACIÓN DANE|      AREA_TOTAL_KM2|densidad_poblacional|
+-------------+------------+--------------+--------------------+--------------------+
|        15176|CHIQUINQUIRA|         58356|1.8929729578688523E8|3.082769870399964...|
|        15176|CHIQUINQUIRA|         58356|1.8929729578688523E8|3.082769870399964...|
|        15176|CHIQUINQUIRA|         58726|1.8929729578688523E8|3.102315844285221E-4|
|        15176|CHIQUINQUIRA|         58356|1.8929729578688523E8|3.082769870399964...|
|        15176|CHIQUINQUIRA|         58726|1.8929729578688523E8|3.102315844285221E-4|
|        15176|CHIQUINQUIRA|         58726|1.8929729578688523E8|3.102315844285221E-4|
|        15176|CHIQUINQUIRA|         58356|1.8929729578688523E8|3.082769870399964...|
|        15176|CHIQUINQUIRA|         57935|1.8929729578688523E8|3.060529721735930...|
|        15176|CHIQUINQUIRA|         57935|1.892972957

In [125]:
# Buscar si "BOGOTA, D.C." está en el DataFrame completo
df_completo.filter(F.col('MUNICIPIO').contains("BOGOTA")).select('COD_DEPARTAMENTO', 'COD_MUNICIPIO', 'MUNICIPIO').show(truncate=False)

+----------------+-------------+------------+
|COD_DEPARTAMENTO|COD_MUNICIPIO|MUNICIPIO   |
+----------------+-------------+------------+
|11              |11001        |BOGOTA, D.C.|
|11              |11001        |BOGOTA, D.C.|
|11              |11001        |BOGOTA, D.C.|
|11              |11001        |BOGOTA, D.C.|
|11              |11001        |BOGOTA, D.C.|
|11              |11001        |BOGOTA, D.C.|
|11              |11001        |BOGOTA, D.C.|
|11              |11001        |BOGOTA, D.C.|
|11              |11001        |BOGOTA, D.C.|
|11              |11001        |BOGOTA, D.C.|
|11              |11001        |BOGOTA, D.C.|
|11              |11001        |BOGOTA, D.C.|
+----------------+-------------+------------+

